# **实验介绍**





## **实验背景与目的**

当今，深度学习在图像、自然语言处理和语音领域都发挥着重要作用。小组成员都对语音感兴趣并具备一定数字信号处理知识。结合大作业要求，小组成员决定探究Encoder-Decoder架构模型作为一种降维方法在语音转换上的应用。






## **实验内容**

1. 语音转换简介
2. 基于Autoencoder的语音转换
3. 基于Hubert的语音转换


# **语音转换简介**



## **语音信号**

语音（speech）信号是一种携带人类语言信息的波（wave）。通常，我们直接采集到的是时域信号。一段示例语音及其时域波形如下所示。





In [ ]:
import IPython.display as ipd
from IPython.display import HTML

ipd.display(ipd.Audio('https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/welcome.wav', rate=16000))


![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/welcome_wav.png)

对于这样的信号，可以使用短时傅里叶变换（Short-time Fourier Transform, STFT）将其变换到频域，从而更方便地观察信号的特点和进行处理。例如一个由具有各种频率和相位的余弦信号合成出的信号如下所示。

$$\begin{align}\mathrm{signal} &= 0.5\cos(2\pi\times220t+\frac{\pi}{2})\\&+0.3\cos(2\pi\times440t+\frac{\pi}{4})\\&+0.2\cos(2\pi\times880t+\pi)\\&+0.1\cos(2\pi\times1760t-\frac{\pi}{2})\\&+0.05\cos(2\pi\times3520t+\frac{\pi}{3})\\&+0.03\cos(2\pi\times7040t+\frac{\pi}{6})\end{align}$$

其时域波形如下图所示。
![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/signal_time.png)

经过傅里叶变换后可以得到其频域信号，从中我们可以清晰地看到该信号是由各个成分的频率与幅值（magnitude），例如频率为7040Hz的信号。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/signal_freq.png)

通常，我们不会对整个语音信号直接做傅里叶变换，因为这样无法得到有用的信息。由于语音具有短时性，也就是在发某个某个音时信号时**稳定的**，即频率和幅度变换很小，可以视为周期信号。
于是我们会将整个语音信号分成一个个的帧（frame），并对每一帧做傅里叶变换，再将这些结果按照时间顺序拼接起来，就可以得到语谱图（spectrogram）。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/make_spect.png)

例如上面的语音对应的语谱图如下所示，可以很清晰地看到信号中各个频率成分随时间地变换情况。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/welcome_lin.png)

进一步，人耳对频率的感知是在对数域的。例如我们感觉音高（pitch）升高3个八度，那么频率实际上变为原来的$2^3=8$倍；另一方面，人耳对低频的感知更为灵敏。根据这些特性，可以刚刚得到的（线性）语谱图上进一步得到梅尔语谱图（mel-spectrogram）。其特点是低频部分分辨率高，而高频部分分辨率低。**梅尔语谱图是语音信号处理中常用的特征**。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/welcome_mel.png)

## **语音转换**

语音转换（voice conversion）可以粗略地理解为变声，与图像领域的风格迁移类似。它的任务是将一将一位**源说话人（source speaker）**的语音转换成**目标说话人（target speaker）**的语音，同时尽可能地保留原始语音的语音**内容**和**情感表达**。



In [ ]:
print('Source speaker:')
ipd.display(ipd.Audio('https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/welcome.wav', rate=16000))

print('Target speaker:')
ipd.display(ipd.Audio('https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/target.wav', rate=16000))
print('Converted:')

ipd.display(ipd.Audio('https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/welcome_converted.wav', rate=22050))

Source speaker:


Target speaker:


Converted:


源语音与转换语音的波形与梅尔语谱图的对比如下。波形差别不大，语谱图有明显差异。我们知道女性的音高通常高于男性，这一点就清楚地反映到了语谱图上：**转换语音频率整体比源语音要高**。

|   source   |  converted    |
| ---- | ---- |
|  ![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/welcome_wav.png)    |  ![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/welcome_converted_wav.png)    |
|   ![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/welcome_mel.png)   |   ![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/welcome_converted_mel.png)   |


## **难点** 

在风格迁移任务中，我们通常认为要任务对象具有两个特征：**内容（content）**与**风格（style）**。而风格迁移的任务则是在保证内容不变的情况下改变风格。

在语音转换中，内容主要是**语义信息（linguistic information）**；而风格主要是**说话人信息（speaker information）**，如音色。不论是波形还是语谱图，它们都同时带有内容信息与说话人信息。因此，语音转换的难点就在于如何做到二者的**解耦（disentanglement）**。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/disentanglement.png)



# **基于Autoencoder的语音转换**

## **Autoencoder简介**

Autoencoder是一种无监督学习模型，通常用于数据的降维、特征提取和数据重构等任务。它由两部分组成：
- 编码器（encoder）
- 解码器（decoder）

编码器将输入数据映射到低维度的潜在空间表示，通常称为编码（encoding）；解码器将编码映射回原始数据的表示，通常称为解码（decoding）。在训练过程中，Autoencoder的目标是尽可能地重构输入数据，即最小化输入数据与解码数据之间的差异。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/auto_model.png)  


## **如何解耦**

使用Autoencoder实现解耦的关键就是**降维**。如果认为输入特征（如梅尔语谱图）包括了内容与说话人信息，那么我们期待：
1. 编码器能将输入进行压缩，使其仅包含内容；
2. 解码器根据编码器的输出与额外的说话人信息重建输入。

模型训练时的架构如下图所示。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/auto_train.png) 

为什么这样有用呢？如果中间的低维向量的维度设置得恰到好处，那么：
1. 如果编码器会输出一些说话人信息，那么这些信息会挤占部分内容信息，从而导致编码器无法完美重建输入；
2. 另一方面，由于编码器已经有额外的说话人信息输入，那么它不需要编码器的输出包含说话人信息。

模型推理时的架构如下图所示。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/auto_infer.png) 



## **如果获取额外的说话人信息**

我们需要一个模型，它的功能与Autoencoder中的编码器相反，即删除内容信息并仅保留说话人信息。同时，其还需要做到对于同一个说话人的不同语音，对应的说话人信息向量应该越接近越好。

能完成这一任务的模型通常取自**说话人识别（speaker recognition）**模型。其任务是将声音信号与特定说话人的身份或属性相关联的技术。它旨在从语音信号中提取出说话人的特征，例如说话人的声调、音色、语速等，并将其用于识别和验证说话人的身份。

这一说话人信息通常称为说话人嵌入（speaker embedding）。

例如，对于下面三段语音，我们可以分别提取speaker embedding，并计算余弦相似度。

|  Audio   |  Audio    |  Cosine Similarity    |
| ---- | ---- | ---- |
| 1  |  2  |  0.41726196  |
| 1  |  3   | 0.06778678   |
| 2  |  3  | 0.0334684 |



In [ ]:
print('Audio1:')
ipd.display(ipd.Audio('https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/welcome.wav', rate=16000))

print('Audio2:')
ipd.display(ipd.Audio('https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/data-science.wav', rate=16000))

print('Audio3:')
ipd.display(ipd.Audio('https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/welcome_converted.wav', rate=22050))


Audio1:


Audio2:


Audio3:


## **实验结果**


### **实验一：转换效果评测**

一些语音转换结果如下所示。从左到右分别是source, target和converted。


In [ ]:
audio_html = '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p225xp225.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p228xp228.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p225xp228.wav"></audio> <br>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p256xp256.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p225xp225.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p256xp225.wav"></audio> <br>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p270xp270.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p256xp256.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p270xp256.wav"></audio> <br>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p270xp270.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p225xp225.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p270xp225.wav"></audio> <br>'
HTML(audio_html)

### **实验二：解耦效果探究**

上面的转换效果虽然感觉不错，但我们并不知道模型是否真正学到了将内容与说话人信息分开。因此，本次实验会修改speaker embedding并观察其对转换结果的影响。

修改的方法是将source与target的embedding进行融合，即取加权平均。

第一种是直接取平均。我们期待输出音频的音色应该介于source与target之间。
$$emb = 0.5 \times src\_emb + 0.5 \times trg\_emb$$

In [ ]:
audio_html = '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p225xp225.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p270xp270.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp2/p225xp270_0.5_1.0.wav"></audio> <br>'

HTML(audio_html)

如果想让结果更接近source，则给source embedding更大的权重。

$$emb = 0.7 \times src\_emb + 0.3 \times trg\_emb$$

（这里可以发现结果与source几乎没有区别，可能是模型具有一定的稳定性。）

In [ ]:
audio_html = '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p225xp225.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p270xp270.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp2/p225xp270_0.7_1.0.wav"></audio> <br>'

HTML(audio_html)

$$emb = 0.2 \times src\_emb + 0.8 \times trg\_emb$$

In [ ]:
audio_html = '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p225xp225.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p270xp270.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp2/p225xp270_0.2_1.0.wav"></audio> <br>'

HTML(audio_html)

但是如果输入一个模型没见过的speaker embedding，输出的结果并不理想。可以从中听出几个问题：
1. 输出的音色不对；
2. 部分发音不准确甚至丢失。

可以猜测，对于1， 其可能原因一方面是speaker encoder没有很好地提取speaker embedding；另一方面则是解码器泛化能力不强，不能准确根据没见过地speaker embedding来重建语音。

对于2，其可能原因是编码器没有正确提取内容信息。



In [ ]:
audio_html = '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp1/p225xp225.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/welcome.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/audios/auto_exp2/ood.wav"></audio> <br>'

HTML(audio_html)

## **实验结论**

1. Autoencoder的架构可以使用无监督学习的方法实现语音转换；
2. 编码器的降维能够比较有效地实现内容与说话人信息的解耦；
3. 仅仅依靠该架构仍然无法非常好地去除说话人信息，**需要其他训练方法、模型架构以及约束**。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/auto_model.png)  




# **基于Hubert的语音转换**

## **Hubert简介**

Hubert（HuBERT）是一种基于Transformer架构的开源语音识别和语音转换模型，它在语音相关任务中取得了显著的成果。与传统的语音模型相比，Hubert采用了自监督学习的方法，通过预测声学特征的局部上下文信息来训练模型。这种自监督学习的方式使得Hubert能够学习到更丰富、更抽象的语音表示，从而在语音识别和语音转换任务中取得更好的性能。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/images/8280764436da68cac874a3b111e747d.png)  

## **如何提取内容信息**
Hubert能够分离内容与说话者细节的能力与以下方面有关：

Hubert采用了深度神经网络架构，具有多层卷积神经网络（CNN）和自注意力机制（self-attention）。这种架构能够从语音信号中提取不同级别的特征表示，包括低级的声学特征和高级的语义特征。通过层级的表示学习，Hubert能够分离出语音信号中的内容信息和说话者细节。

Hubert通过自监督学习的方式进行训练。在训练过程中，它使用Masking（掩码）操作来隐藏部分输入语音的信息，通过让模型预测被掩码的部分来促使模型学习更具语义含义的特征表示。

Hubert使用K-means聚类算法来提取语音特征表示。通过对大量语音样本进行聚类，可以得到一组中心向量，这些中心向量用于计算输入语音样本与其之间的距离，提供更紧凑且有判别性的特征表示。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/images/b56483bcf5988d1b72a236f47d60c5e.png) 

可以简单理解为模型通过“听”大量的语音，发现了其中的规律。它讲类似的语音片段归到同一类，创造了一种独有的“语言”。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/unit.png) 

## **模型设计**

整个模型主要分为两个部分：
1. 基于Hubert的内容编码器
2. 基于HiFiGAN的解码器

该模型仍然属于Encoder-Decoder架构，但其输入是波形而非梅尔语谱图。另一方面，HuBert采用的是开源模型，因此只需要训练解码器。

此外，HiFiGAN本身是一个基于深度学习的声码器（vocoder），用于将梅尔语谱图重建成波形。它具有以下优势：
1. 采用对抗训练，使得输出的语音更加接近人声
2. 有多个判别器，可以有效捕捉各个频率的特征
3. 推理速度快


由于此处我们使用的是Hubert的输出，因此需要修改其输入的维度以及用于上采样的参数。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/master/images/hubert_hifigan.png) 





## **实验结果**

### **实验一：转换效果评测**

In [ ]:
print('Target Speaker:')
ipd.display(ipd.Audio('https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/audios/0-target.wav', rate=16000))


Target Speaker:


In [ ]:
audio_html = '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/audios/s2.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/audios/svc2.wav""></audio><br>' 
HTML(audio_html)

In [ ]:
audio_html = '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/audios/s3.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/audios/svc3.wav""></audio><br>' 
HTML(audio_html)

In [ ]:
audio_html = '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/audios/s1.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/audios/svc1.wav""></audio><br>' 
HTML(audio_html)

以上面的source和target音频为例，分析他们的波形和频谱图，可以看到source在经过模型降维和重建之后的音频质量较好。

|   source   |  converted    |
| ---- | ---- |
|  ![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/images/4b354ef495ae5dadd1179def11877d8.png)    |  ![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/images/c3c4c66d567d13982bf9a551e5064f6.png)   

**中文语音转换（unseen speaker）**

In [ ]:
audio_html = '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/audios/test1.wav"></audio>' + \
        '<audio controls src="https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/audios/svc4.wav""></audio><br>' 
HTML(audio_html)

### **实验二：解耦效果探究**
在 Hubert 模型中，音频数据首先通过一系列的卷积层和自注意力层进行处理，这些层可以捕捉到音频数据中的各种模式和特征。然后，这些特征被聚类成一系列的离散语音单元，这就是输出units。为了方便观测结果，我们让对不同语者说同一句话得到的音频用Hubert处理，得到speech units进行对比：

In [ ]:
print('Female speaker:')
ipd.display(ipd.Audio('https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/audios/F.wav', rate=16000))

print('Male speaker:')
ipd.display(ipd.Audio('https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/audios/M.wav', rate=16000))


Female speaker:


Male speaker:


首先，如果我们直接让两个units相减，其结果如下所示。可以发现图中大部分值都接近于0，说明不同语者在说相同话的情况下，得到的不同units在很多维度上的差距很小甚至没有，说明经过hubert得到的units向量可以成功提取出更高层的语义信息，而其中的差异即说话人信息的内容只占了一小部分。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/images/NU_1YNM8D333F0Z___45F58.png)

考虑到同一句话还可以用不同语速说出，直接相减可能不是很准确。于是，先采用动态时间规整（Dynamic Time Warping， DTW）来对齐两个units。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/images/b8c1fff49b8a6ba12135484420f7634.png) 

然后根据对齐得到的路径，求对应帧向量的相似度。

![image](https://cslabcg.whu.edu.cn/vdir/Gitlab/2020300004044/data-science-resources/-/raw/patch-1/images/dtw_sim.png)|
    


## **实验结论**

1. 基于Hubert的编码器架构可以成功降维并分离出说话人和语义信息
2. 基于Hubert和HifiGAN的模型可以合成高质量的目标说话人波形
3. 存在一定的发音错误和语调不自然
，**需要进一步优化**。




# **参考**

https://arxiv.org/abs/1905.05879

https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9746484


https://www.youtube.com/watch?v=sWz4e-DM4JU&t

http://speech.ee.ntu.edu.tw/DSP2021Spring/

https://github.com/jaywalnut310/vits

https://github.com/jik876/hifi-gan

https://github.com/bshall/soft-vc